File to import, clean, and analyze data. Can then move the code to a different file for proper documentation purposes.


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [12]:
# URL for the dataset
url = "https://github.com/JeffSackmann/tennis_atp/blob/master"

# Load the dataset, going from 2004 to 2024 (picking only atp_matches_years.csv)
years = list(range(2004, 2025))
matches = []
for year in years:
    file_url = f"{url}/atp_matches_{year}.csv?raw=true"
    df = pd.read_csv(file_url)
    matches.append(df)
# Concatenate all dataframes into one
matches_df = pd.concat(matches, ignore_index=True)
# Drop duplicates
# matches_df.drop_duplicates(inplace=True)
# Reset index
matches_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the dataframe
print(matches_df.head())
# Display the shape of the dataframe
print(matches_df.shape)
# Display the columns of the dataframe
print(matches_df.columns)
# Display the data types of the columns
print(matches_df.dtypes)
# Display the summary statistics of the dataframe
print(matches_df.describe())

  tourney_id    tourney_name surface  draw_size tourney_level  tourney_date  \
0  2004-1536  Madrid Masters    Hard         48             M      20041018   
1  2004-1536  Madrid Masters    Hard         48             M      20041018   
2  2004-1536  Madrid Masters    Hard         48             M      20041018   
3  2004-1536  Madrid Masters    Hard         48             M      20041018   
4  2004-1536  Madrid Masters    Hard         48             M      20041018   

   match_num  winner_id  winner_seed winner_entry  ... l_1stIn l_1stWon  \
0          1     102610          NaN          NaN  ...    64.0     40.0   
1          2     103344          NaN          NaN  ...    29.0     21.0   
2          3     102179          NaN            Q  ...    28.0     23.0   
3          4     103084          NaN          NaN  ...    53.0     35.0   
4          5     104417          NaN          NaN  ...    37.0     27.0   

   l_2ndWon l_SvGms  l_bpSaved  l_bpFaced  winner_rank winner_rank_points 